In [ ]:
import random
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
def create_neural_network(params):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(params['hidden_layer_units'], activation='relu',
                              input_shape=(params['input_shape'],)),
        tf.keras.layers.Dense(params['output_shape'], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [3]:
def generate_initial_population(population_size, parameter_ranges):
    population = []
    for _ in range(population_size):
        params = {
            'hidden_layer_units': random.randint(parameter_ranges['hidden_layer_units'][0],
                                                 parameter_ranges['hidden_layer_units'][1]),
            'input_shape': parameter_ranges['input_shape'],
            'output_shape': parameter_ranges['output_shape']
        }
        population.append(params)
    return population


def fitness_function(model, X_train, y_train):
    _, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return accuracy

In [4]:
def tournament_selection(population, fitness_values, tournament_size):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(range(len(population)), tournament_size)
        winner = tournament[0]
        for competitor in tournament[1:]:
            if fitness_values[competitor] > fitness_values[winner]:
                winner = competitor
        selected.append(population[winner])
    return selected


def crossover(parent1, parent2):
    child = {
        'hidden_layer_units': random.choice([parent1['hidden_layer_units'],
                                             parent2['hidden_layer_units']]),
        'input_shape': parent1['input_shape'],
        'output_shape': parent1['output_shape']
    }
    return child


def mutate(params, mutation_rate, parameter_ranges):
    mutated_params = params.copy()
    for key in mutated_params:
        if key == 'hidden_layer_units':
            if random.random() < mutation_rate:
                min_val, max_val = parameter_ranges[key]
                mutated_params[key] = random.randint(min_val, max_val)
    return mutated_params

In [5]:
def genetic_algorithm(X_train, y_train, population_size, generations,
                      mutation_rate, tournament_size, parameter_ranges):
    population = generate_initial_population(population_size, parameter_ranges)
    for _ in range(generations):
        models = [create_neural_network(params) for params in population]

        for model in models:
            model.fit(X_train, y_train, epochs=20, verbose=0)

        fitness_values = [fitness_function(model, X_train, y_train) for model in models]

        selected = tournament_selection(population, fitness_values, tournament_size)

        offspring = []
        for i in range(0, population_size, 2):
            parent1 = selected[i]
            parent2 = selected[i + 1]
            child = crossover(parent1, parent2)
            offspring.append(mutate(child, mutation_rate, parameter_ranges))
            offspring.append(mutate(child, mutation_rate, parameter_ranges))

        population = offspring
        print("Best Accuracy:", max(fitness_values))

    best_model_params = max(population,
                            key=lambda x: fitness_function(create_neural_network(x),
                                                           X_train, y_train))
    return best_model_params

In [6]:
from sklearn.datasets import load_wine
data = load_wine()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(X_train.shape)

(142, 13)


In [ ]:
population_size = 40
generations = 20
mutation_rate = 0.1
tournament_size = 5
parameter_ranges = {
    'hidden_layer_units': [10, 100],
    'input_shape': X_train.shape[1],
    'output_shape': len(np.unique(y_train))
}

best_params = genetic_algorithm(X_train, y_train, population_size, generations,
                                mutation_rate, tournament_size, parameter_ranges)

In [22]:
best_model = create_neural_network(best_params)
best_model.fit(X_train, y_train, epochs=50, verbose=0)
best_fitness = fitness_function(best_model, X_train, y_train)

print("Best Parameters:", best_params)
print("Best Accuracy:", best_fitness)

Best Parameters: {'hidden_layer_units': 99, 'input_shape': 13, 'output_shape': 3}
Best Accuracy: 0.9154929518699646
